In [1]:
import pandas as pd
import os as os
import numpy as np
import matplotlib.pyplot as plt
import sys
from transformers import pipeline

In [2]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

KeyboardInterrupt: 

Reading in the datasets

In [3]:
bbb_main_businesses = pd.read_csv("/Users/christopherwang/Desktop/BBB_DataThink_2/Datasets/Main_BusinessDataBBB_02022024.csv")
bbb_TOB_Rollup = pd.read_csv("/Users/christopherwang/Desktop/BBB_DataThink_2/Datasets/bbb_type_of_business_rollup.csv")

In [35]:
bbb_main_businesses = bbb_main_businesses.query("`State` == 'GA'")

Pulling the Categories & Identifying the Unique Self-Selected Business Categories

In [37]:
labels = ['Agriculture, Forestry, Fishing and Hunting', 'Mining, Quarrying, and Oil and Gas Extraction', 'Utilities', 'Construction', 
          'Manufacturing', 'Wholesale Trade', 'Retail Trade', 'Transportation and Warehousing', 'Information, includes publishing industries, motion picture and sound recording, broadcasting, telecommunications, as well as data processing and hosting', 'Finance and Insurance', 'Real Estate and Rental and Leasing',
          'Professional, Scientific, and Technical Services', 'Management of Companies and Enterprises', 'Administrative and Support and Waste Management and Remediation Services'
          ,'Educational Services', 'Health Care and Social Assistance', 'Arts, Entertainment, and Recreation', 'Accommodation and Food Services', 
          'Other Services (except Public Administration)', 'Public Administration']

In [38]:
labels2 = [
    'Natural Resources and/or Mining',
    'Manufacturing',
    'Construction',
    'Trade, Transportation, and/or Utilities',
    'Information', 'Financial Activities', 'Professional and/or Business Services', 'Education and/or Health Services', 'Leisure and/or Hospitality', 'Other Services (except public administration)'
]

In [39]:
labels_to_be_processed = np.where(bbb_main_businesses['TOB'].isna(), bbb_main_businesses['Business'], bbb_main_businesses['TOB'] )
labels_to_be_processed = pd.Series(labels_to_be_processed)
labels_to_be_processed = labels_to_be_processed.rename("TOB")
labels_to_be_processed = labels_to_be_processed.drop_duplicates()

In [28]:
#classifier('Construction Reports', labels, multi_label=True)

In [40]:
test_labels = labels_to_be_processed.sample(n = 20)

In [34]:
def zeroshot(dataset, labels):
    """
    This function takes in a dataset with a text column and the corresponding support labels of the standardized label.
    The Zero-Shot Topic Classification algorithm will determine a confidence score for each support label.
    The corresponding confidence score given by the model will be added as a new column to the original dataset.
    """
    result = []
    for i in dataset:
        result.append(classifier(i, labels))
    return result

# apply the zero shot topic classification algorithm
df4 = zeroshot(test_labels, labels)

In [10]:
# Initialize lists to store data
sequences = []
max_scores = []
max_labels = []

# Iterate over the list of dictionaries
for d in df4:
    sequence = d['sequence']
    label = d['labels'][d['scores'].index(max(d['scores']))]
    score = max(d['scores'])
    
    # Append data to lists
    sequences.append(sequence)
    max_scores.append(score)
    max_labels.append(label)

# Create DataFrame from lists
result = pd.DataFrame({'sequence': sequences, 'max_score': max_scores, 'max_label': max_labels})


In [11]:
result

,sequence,max_score,max_label
0,General Contractor,0.823069,Construction
1,Rentals by Owner,0.282339,Real Estate and Rental and Leasing
2,Junk Removal,0.365192,Administrative and Support and Waste Managemen...
3,Credit Card Merchant Services,0.113278,Information
4,Flooring Contractors,0.738422,Construction
...,...,...,...
95,Textile Brokers,0.127252,Other Services (except Public Administration)
96,Relocation Services,0.122538,Transportation and Warehousing
97,Moving Companies,0.322646,Management of Companies and Enterprises
98,Gift Shop,0.877510,Retail Trade


In [ ]:
result.to_csv("/Users/christopherwang/Desktop/BBB_DataThink_2/Datasets/NPAIS_Classification_Groupings.csv")